In [1]:
from imutils import paths
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json
from tensorflow.keras.models import load_model
import tensorflow as tf

## LOAD model

In [2]:
model = load_model('output_models/mask_detector_mobilenet_v2')
model.summary()
# The export path contains the name and the version of the model
tf.keras.backend.set_learning_phase(0)  # Ignore dropout at inference
export_path = './serving_models/mobilenet_v2/1'

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/keras/backend.py:401: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


## Export model


In [3]:
model.save(export_path)

INFO:tensorflow:Assets written to: ./serving_models/mobilenet_v2/1/assets


/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [5]:
# simple_save
with tf.compat.v1.keras.backend.get_session() as sess:
    tf.compat.v1.saved_model.simple_save(
        sess,
        export_path,
        inputs={'input_image': model.input},
        outputs={'y_pred': model.output})

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./serving_models/mobilenet_v2/1/saved_model.pb


## saved_model_cli

In [4]:
# CHECK saved_model
# i.e: version 1
!saved_model_cli show --dir serving_models/mobilenet_v2/1 --tag_set serve --signature_def serving_default


2022-03-18 03:49:54.203184: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.6/dist-packages/cv2/../../lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-03-18 03:49:54.203212: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The given SavedModel SignatureDef contains the following input(s):
  inputs['input_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_input_1:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


## gRPC (Google Remote Procedures Calls) vs RESTful (Representational State Transfer)

In [5]:
!echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -

!apt-get update && apt-get install tensorflow-model-server

deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  18055      0 --:--:-- --:--:-- --:--:-- 18055
OK
Hit:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
G

## Thực hiện khởi chạy 

In [ ]:
!tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=face-mask-detection-serving --model_base_path=/home/dev-fti/thuc/Face-Mask-Detection/serving_models/mobilenet_v2

2022-03-18 03:50:02.206656: I tensorflow_serving/model_servers/server.cc:89] Building single TensorFlow model file config:  model_name: face-mask-detection-serving model_base_path: /home/dev-fti/thuc/Face-Mask-Detection/serving_models/mobilenet_v2
2022-03-18 03:50:02.206863: I tensorflow_serving/model_servers/server_core.cc:465] Adding/updating models.
2022-03-18 03:50:02.206875: I tensorflow_serving/model_servers/server_core.cc:594]  (Re-)adding model: face-mask-detection-serving
2022-03-18 03:50:02.307472: I tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: face-mask-detection-serving version: 1}
2022-03-18 03:50:02.307510: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: face-mask-detection-serving version: 1}
2022-03-18 03:50:02.307544: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: face-mask-detection-serving version: 1}
2022-03-18 03:50:02.307608: I exter